# Introducción: nubes de puntos en Python

Antes de proseguir, veamos como funcionan estas celdas. Ahora mismo estás leyendo una celda de texto. Si haces doble clic en ella, puedes acceder al "código" que se usa para generar el texto. Arriba a la derecha de cada celda tienes opciones para agregar celda, borrar, cambiar tipo de celda y ejecutar (en el caso de celdas con código). Si ejecutas una celda aparecera su lateral izquierdo en verde hasta que finalice la ejecución y aparecerá un número al lado entre corchetes con el orden en el que se ejecutó la celda y debajo cualquier elemento que devuelva el código o quieras imprimir por pantalla.

Los comentarios del código se muestran despues de una almohadilla (los comentarios son código no ejecutable)

Las modificaciones se guardan automáticamente en el notebook. 

En la parte izquierda de la pantalla tienes las opciones de añadir archivos, librería, lista de variables o contenido. En la parte superior diversas opciones para modificar el notebook, agregar archivos y configurar la ejecución del código.

In [1]:
# Ejecutame
print("Hola Mundo!")

Hola Mundo!


# Datos


 Para poder trabajar con datos, antes debemos cargarlos en el notebook desde nuestro disco duro. Para cargar nuestras nubes de puntos, seleccionamos *Attached files* a la derecha, *Upload*, y *Upload folder*. Después seleccionamos la carpeta en nuestro disco duro donde tenemos cargadas las nubes, nos pedirá una confirmación y cargamos la carpeta. De esta forma, todo el contenido de la carpeta se carga a la vez. También se puede arrastrar la carpeta de nuestro disco duro a *Attached files*.

 Para comprobar que la carga fue correcta, en el navegador de *Attached files*, hay un pequeño botón de refrescar, cuando hacemos clic paraece la carpeta Nubes, y dentro las nubes en formato pcd.
 

# Librerías

Por defecto, Datalore y python tiene instalados una serie de librerías para emplear. Python es un lenguaje muy básico, casi para realizar cualquiera operaciones tenemos librerías que nos ayudan a ejecutarla sin tener que programarla.

No obstante, la librería para trabajar con nubes de puntos no se agrega a nuestro entorno de trabajo por defecto y tenemos que agregarla. Para hacerlo, vamos a la pestaña de *Librerías*, y en *Explore*, buscamos:

<center> pyntcloud </center>

Puedes encontrar la ayuda de esta librería en:
- https://pyntcloud.readthedocs.io/en/latest/#

Nos lista una opción, la seleccioanmos ay aparece una pestaña nueva con información de la librería. Le damos a *Install*. Esperamos a que termine y, si todo fue bien, aparecerá una notificación en verde indicándolo. 

Ejecuta la siguiente celda, en ella están las librerías que se deben importar y que vamos a usar en el código. Para importar las librerías, estas deben estar instaladas en el entorno de trabajo. Acabamos de isntalar *pyntcloud* y *numpy* está instalada por defecto. *Numpy* es una librería para realizar operaciones matemáticas y estructurar datos.

In [2]:
# Importar librerías
import numpy as np
from pyntcloud import PyntCloud

# Objetivos

Una vez que las librerías están cargadas, en esta práctica vamos a ver fragmentos de código que expliquen cómo funciona el procesado automático de nubes de puntos realizado internamente por los vehículos autónomos. Las operaciones que se estudiarán a continuación son:

- Lectura y escritura de nubes de puntos
- Cálculo de vecinos
- Estimación de normales
- Generación de nuevos atributos
- Reducción del número de puntos
- Detección de planos en las nubes

# Lectura y escritura de nubes de puntos

El primer paso para procesar las nubes es cargarlas en el programa, o leerlas. El comando para leer las nubes está en la siguiente celda. Existen muchas extensiones para las nubes de puntos txt, ply, pcd, xyz, bin... Cada librería permite la lectura de algunos de ellos. En nuestro caso, las nubes de Velodyne proporcionadas en el kitti dataset se encuentran en formato bin. Como el formato bin no es extensamente admitido por muchos programas (como CloudCompare) y librerías, las nubes fueron transformadas a formato pcd.

El comando de lectura solo necesita la dirección y el nombre de la nube de puntos a leer. La nube se guarda en una variable y, mediante print, podemos ver las características de la nube cargada.

In [3]:
# Lectura de la nube de puntos
cloud = PyntCloud.from_file("Nubes/000036.pcd")
print(cloud)

PyntCloud
122749 points with 0 scalar fields
0 faces in mesh
0 kdtrees
0 voxelgrids
Centroid: -0.884735107421875, 1.1438301801681519, -1.0953199863433838
Other attributes:



Se listan algunas características de la nube de puntos, como puede ser su número, si tienen o no atributos (0 campos escalares en nuestro caso), referencias a mesh, kdtrees y voxeles (que son formas de estructurar la nube) y su centroide. La matriz de  puntos se puede ver mediante el siguiente script.

Si te fijas, también se puede visualizar los datos sin llamar a *print*, simplemente escribiendo la variable a mostrar. Si usas esta opción se muestran los datos estructurados de forma más amigable, pero se sobreescribe los datos anteriores, es decir, solo se muestra la última variable llamada, no como con *print*.

In [4]:
# Acceso a los puntos
cloud.points

,x,y,z
0,73.017906,7.409195,2.689679
1,72.119446,7.546182,2.660625
2,71.842979,7.746161,2.651559
3,72.741516,8.074124,2.683469
4,72.790054,8.311100,2.685393
...,...,...,...
122744,3.750455,-1.423028,-1.731100
122745,3.744989,-1.408038,-1.726092
122746,3.755518,-1.399045,-1.729087
122747,3.774050,-1.393048,-1.737084


Una vez que sabemos dónde están los puntos en nuestra variable, vamos a proceder la exportación a disco de la nube, o al guardado de la nube. De momento nuestra nube solo tiene coordenadas geométricas. La librería *pyntcloud* no permite escribir todas las extensiones de las nubes de puntos, por lo tanto vamos a recurrir una escritura mediante la librería *numpy*.

La función savetxt nos permite guardar matrices de datos en un txt. Los ficheros txt son leíbles por multitud de programas, como programas GIS, Excel, otras librerías de Python o Cloud Compare. La función savetxt necesita el nombre y la ruta para guardar la nube, la matriz de puntos y un delimitador para separar los atributos de cada punto en cada línea.

El código de escritura se muestra a continuación.

In [5]:
# Escritura de la nube de puntos
ruta = "Nubes/new_cloud.txt"
puntos = cloud.points
np.savetxt(ruta,puntos,delimiter=',') 

Si accedemos al directorio Nubes donde están guardadas las nubes 000036.pcd y new_cloud.txt, se puede observar que el peso en disco de cada nube es distinto debido al formato de compresión de cada tipo de datos. Cuando abrimos ambas nubes con CloudCompare, vemos que las nubes coinciden correctamente y tienen el mismo número de puntos. Esto es correcto, pues solo hemos abierto la nube y vuelto a guardar.

Para descargar una nube a nuestro ordenador, tenemos que seleccionarla y darle a descargar.

# Cálculo de vecinos

El cálculo de vecinos es una de las operaciones fundamentales del procesado con nubes de puntos. Un punto aislado no presenta características geométricas distintivas, por lo tanto, es necesario conocer la posición de este punto respecto a sus vecinos. Si lo comparamos con el procesado de imagen, dado un píxel I_nm, sus píxeles vecinos se obtienen a partir de los índices vecinos, por ejemplo, n+1 y m-1. La búsqueda de puntos vecinos por índices vecinos no es aplicable en nubes, un punto situado en la línea siguiente a otro no tiene por qué ser el más próximo. Puedes comprobarlo en CloudCompare seleccionando puntos próximos y viendo su índice. 

Por suerte, existe una función que realiza la operación de calcular vecinos (siguiente celda). En ella solo es preciso definir cuántos vecinos queremos calcular para cada punto y definir la nube sobre la que calculamos los vecinos. El número de vecinos *k* influye en el cálculo de características. Un número bajo implica que las características son más locales y sensibles al ruido, uno alto implica menor localidad y sensibilidad al ruido, pero las características pueden quedar menos definidas y el tiempo de procesado es mayor. Los *k* más comunes son 5, 10, 25 y 50.

El resultado del cálculo los vecinos devuelve una matriz de índices donde en cada fila se corresponde a cada punto de la nube, y cada columna el índice de su vecino más cercano por orden.

In [6]:
# Calculo de vecinos
k_neighbors_10 = cloud.get_neighbors(k=10)
print(k_neighbors_10)

[[     3      1      4 ...      6   4096   4098]
 [     2   2060      3 ...   4096   4098      6]
 [     1   2060      3 ...      0   4098      6]
 ...
 [122745 122747 122744 ... 122740 121601 122739]
 [122748 122746 122745 ... 122741 121600 121602]
 [122747 122746 122745 ... 121603 121602 121600]]


# Estimación de normales

La estimación de normales es otra de las operaciones comunes en la nube de puntos. La normal de cada punto define su orientación respecto a sus vecinos. Esta operación permite conocer qué puntos pertenecen a áreas horizontales, inclinadas y verticales de la nube, y es un paso intermedio para obtener otros atributos. La normal de cada punto se compone de tres componentes (x,y,z), orientaciones respecto a cada eje. 

La función para calcular las normales necesita como input los índices vecinos de cada punto. Como resultado, la función añade a nuestra nube la información de las normales identificada según el número de vecinos empleado.

In [7]:
# Cálculo de normales
cloud.add_scalar_field("normals", k_neighbors=k_neighbors_10)
cloud.points

,x,y,z,nx(11),ny(11),nz(11)
0,73.017906,7.409195,2.689679,-0.557236,0.318483,0.766849
1,72.119446,7.546182,2.660625,-0.557236,0.318483,0.766849
2,71.842979,7.746161,2.651559,-0.557236,0.318483,0.766849
3,72.741516,8.074124,2.683469,-0.557236,0.318483,0.766849
4,72.790054,8.311100,2.685393,-0.557236,0.318483,0.766849
...,...,...,...,...,...,...
122744,3.750455,-1.423028,-1.731100,0.407254,-0.141298,0.902319
122745,3.744989,-1.408038,-1.726092,0.407253,-0.141298,0.902319
122746,3.755518,-1.399045,-1.729087,-0.063138,0.073016,0.995330
122747,3.774050,-1.393048,-1.737084,-0.058267,0.072487,0.995666


# Tarea 1

Por supuesto, la estimación de las normales depende del número de vecinos. Usando los scripts de cálculo de vecinos y normales, implementa en la celda siguiente un cálculo de normales basada en 25 vecinos para ver si cambian los valores de cada componente nx, ny y nz. 

In [8]:
# Cálculo de vecinos para k = 25
k_neighbors_25 = COMPLETAR

# Estimación de normales
cloud.add_scalar_field("normals", k_neighbors=k_neighbors_25)

#Visualización de resultado
cloud.points

,x,y,z,nx(11),ny(11),nz(11),nx(26),ny(26),nz(26)
0,73.017906,7.409195,2.689679,-0.557236,0.318483,0.766849,-0.457854,0.333181,0.824233
1,72.119446,7.546182,2.660625,-0.557236,0.318483,0.766849,-0.357278,0.361503,0.861202
2,71.842979,7.746161,2.651559,-0.557236,0.318483,0.766849,-0.357278,0.361503,0.861202
3,72.741516,8.074124,2.683469,-0.557236,0.318483,0.766849,-0.530865,0.338234,0.777033
4,72.790054,8.311100,2.685393,-0.557236,0.318483,0.766849,-0.530865,0.338234,0.777033
...,...,...,...,...,...,...,...,...,...
122744,3.750455,-1.423028,-1.731100,0.407254,-0.141298,0.902319,-0.049804,0.041143,0.997911
122745,3.744989,-1.408038,-1.726092,0.407253,-0.141298,0.902319,-0.046452,0.050340,0.997651
122746,3.755518,-1.399045,-1.729087,-0.063138,0.073016,0.995330,-0.053183,0.033867,0.998010
122747,3.774050,-1.393048,-1.737084,-0.058267,0.072487,0.995666,-0.054668,0.033956,0.997927



Como se puede ver, los valores entre n(11) y n(26) cambian. Pero es más interesante analizar como esto afecta a nuevos atributos calculados a partir de estos datos, como puede ser la inclinación. En la siguiente celda se añade el valor de inclinación de cada punto respecto a sus 10 vecinos más próximos.

In [9]:
# Para añadir la inclinación, la función necesita el campo por defecto de las normales de la nube
# La siguiente linea copia los valores de las normales estimadas para 10 vecinos a los valores de las normales por defecto de la nube 
cloud.points[["nx", "ny", "nz"]] = cloud.points[["nx(11)", "ny(11)", "nz(11)"]]

# Se calcula la inclinación
cloud.add_scalar_field("inclination_degrees")

# Se muestran los resultados de la nube
cloud.points

,x,y,z,nx(11),ny(11),nz(11),nx(26),ny(26),nz(26),nx,ny,nz,inclination_deg
0,73.017906,7.409195,2.689679,-0.557236,0.318483,0.766849,-0.457854,0.333181,0.824233,-0.557236,0.318483,0.766849,39.928261
1,72.119446,7.546182,2.660625,-0.557236,0.318483,0.766849,-0.357278,0.361503,0.861202,-0.557236,0.318483,0.766849,39.928257
2,71.842979,7.746161,2.651559,-0.557236,0.318483,0.766849,-0.357278,0.361503,0.861202,-0.557236,0.318483,0.766849,39.928257
3,72.741516,8.074124,2.683469,-0.557236,0.318483,0.766849,-0.530865,0.338234,0.777033,-0.557236,0.318483,0.766849,39.928261
4,72.790054,8.311100,2.685393,-0.557236,0.318483,0.766849,-0.530865,0.338234,0.777033,-0.557236,0.318483,0.766849,39.928261
...,...,...,...,...,...,...,...,...,...,...,...,...,...
122744,3.750455,-1.423028,-1.731100,0.407254,-0.141298,0.902319,-0.049804,0.041143,0.997911,0.407254,-0.141298,0.902319,25.535421
122745,3.744989,-1.408038,-1.726092,0.407253,-0.141298,0.902319,-0.046452,0.050340,0.997651,0.407253,-0.141298,0.902319,25.535412
122746,3.755518,-1.399045,-1.729087,-0.063138,0.073016,0.995330,-0.053183,0.033867,0.998010,-0.063138,0.073016,0.995330,5.539269
122747,3.774050,-1.393048,-1.737084,-0.058267,0.072487,0.995666,-0.054668,0.033956,0.997927,-0.058267,0.072487,0.995666,5.336342


# Tarea 2

Calcula la inclinación para  25 vecinos y agregar ese campo a la nube. 

In [10]:
# Guarda el atributo inclination_deg como inclination_deg_10 y que no se sobreescriba cuando calculemos la inclinación con 25
cloud.points[["inclination_deg_10"]] = COMPLETAR

# Reasigna las variables n(26) al atributo n por defecto
cloud.points[["nx", "ny", "nz"]] = COMPLETAR

# Calcula la inclinación
cloud.add_scalar_field("inclination_degrees")

# Guarda el atributo inclination_deg como inclination_deg_25 para que quede bien organizado
cloud.points[["inclination_deg_25"]] = COMPLETAR

# Visualizar nuevos atributos 
cloud.points

,x,y,z,nx(11),ny(11),nz(11),nx(26),ny(26),nz(26),nx,ny,nz,inclination_deg,inclination_deg_10,inclination_deg_25
0,73.017906,7.409195,2.689679,-0.557236,0.318483,0.766849,-0.457854,0.333181,0.824233,-0.457854,0.333181,0.824233,34.489216,39.928261,34.489216
1,72.119446,7.546182,2.660625,-0.557236,0.318483,0.766849,-0.357278,0.361503,0.861202,-0.357278,0.361503,0.861202,30.548214,39.928257,30.548214
2,71.842979,7.746161,2.651559,-0.557236,0.318483,0.766849,-0.357278,0.361503,0.861202,-0.357278,0.361503,0.861202,30.548222,39.928257,30.548222
3,72.741516,8.074124,2.683469,-0.557236,0.318483,0.766849,-0.530865,0.338234,0.777033,-0.530865,0.338234,0.777033,39.010284,39.928261,39.010284
4,72.790054,8.311100,2.685393,-0.557236,0.318483,0.766849,-0.530865,0.338234,0.777033,-0.530865,0.338234,0.777033,39.010284,39.928261,39.010284
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122744,3.750455,-1.423028,-1.731100,0.407254,-0.141298,0.902319,-0.049804,0.041143,0.997911,-0.049804,0.041143,0.997911,3.703907,25.535421,3.703907
122745,3.744989,-1.408038,-1.726092,0.407253,-0.141298,0.902319,-0.046452,0.050340,0.997651,-0.046452,0.050340,0.997651,3.927701,25.535412,3.927701
122746,3.755518,-1.399045,-1.729087,-0.063138,0.073016,0.995330,-0.053183,0.033867,0.998010,-0.053183,0.033867,0.998010,3.614926,5.539269,3.614926
122747,3.774050,-1.393048,-1.737084,-0.058267,0.072487,0.995666,-0.054668,0.033956,0.997927,-0.054668,0.033956,0.997927,3.689819,5.336342,3.689819


Entre las columnas inclination_deg_10 e inclination_deg_25 se puede observar la diferencia en la inclinación, pero sería mucho mejor poder visualizar estos valores de forma gráfica. Para ello, vamos a exportar la nube de puntos con estos dos nuevos atributos y abrirla en Cloud Compare.

Anteriormente se ha visto como guardar las coordenadas XYZ, pero no como añadir los atributos. El siguiente código complemebnta la exportación concadenando las columnas de atributos deseadas. Si no aparece la nueva nube en el directorio, acuérdate de refrescar.

In [12]:
# Escritura de la nube de puntos con atributos

# Definicion de la ruta y nombre del archivo
ruta = "Nubes/new_cloud_with_atributes.txt"

#Seleccion de datos 
datos = cloud.points[["x", "y", "z","inclination_deg_10","inclination_deg_25"]]

# Guardado
np.savetxt(ruta,datos,delimiter=',') 

# Tarea 3

Exporta la nube de datalore (seleccionándola y descargándola) e importa la nueva nube en Cloud Compare y cambia entre los valores escalares para comprobar cómo afecta el cambio del número de vecinos. 

Como puedes observar, las variaciones son pequeñas, pero pueden suponer una fuente de error en un algoritmo de clasificación. El número de vecinos óptimo cambia dependiendo de la aplicación. Para la selección óptima del número de vecinos se debe testear el comportamiento del algoritmo y seleccionar el número de vecinos que asegure el comportamiento más robusto. Además, también se debe tener en cuenta el tiempo de procesado.

# Reducción del número de puntos

Como en CloudCompare, *pyntcloud* tiene funciones de simplificación:

- Random: Selecciona puntos de forma aleatoria definiendo un número de puntos restante
- Voxel: La voxelización es una reestructuración de la nube en una imagen 3D. La función necesita definir un tamaño de voxel para transformar la nube a imagen. Después, la imagen 3D vuelve a ser transformada a nube de puntos, dejando el punto más cercano al centro de cada voxel.

El siguiente script reduce el número de puntos de forma aleatoria.

In [13]:
# Aplicamos la reducción para dejar la nube con 10k puntos y la guardamos como una nube nueva
cloud_sampled_ran = cloud.get_sample("points_random", n = 10000, as_PyntCloud=True)

# Guardamos en disco
np.savetxt("Nubes/cloud_sample_ran.txt",cloud_sampled_ran.points[["x", "y", "z"]],delimiter=',') 

El siguiente script reduce el número de puntos en base a una distribucion voxel.

In [14]:
# Antes de aplicar la reducción, debemos generar el voxelizado. Emplearemos un voxelizado con voxeles de tamaño 0.5 m
voxelgrid_id = cloud.add_structure("voxelgrid", size_x=0.50, size_y=0.50, size_z=0.50)
voxelgrid = cloud.structures[voxelgrid_id]

# Aplicamos la reducción y la guardamos como una nube nueva
cloud_sampled_vox = cloud.get_sample("voxelgrid_nearest", voxelgrid_id=voxelgrid_id, as_PyntCloud=True)

# Guardamos en disco
np.savetxt("Nubes/cloud_sample_vox.txt",cloud_sampled_vox.points[["x", "y", "z"]],delimiter=',') 

# Tarea 4

Importa ambas nubes reducidas en CloudCompare y comprueba si el número de puntos es correcto en la reducción aleatoria. Comparando ambas nubes, y la original, ¿qué nube tiene una distribuación más regular? 

# Detección de formas

Random sample consensus (RANSAC) es un método iterativo para calcular los parámetros de un modelo matemático de un conjunto de datos observados que contiene valores atípicos. En nubes de puntos es una herramienta muy útil para detectar planos. Los planos suponen una de las principales geometrías que coinciden con las superficies de los elementos construidos.

En la librería *pyntcloud*, RANSAC no devuelve el modelo paramétrico del plano, se implementa de la misma forma que calculando otro atributo nuevo de la nube mediante *add_scalar_field* y lo añade como una nueva columna. La nueva columna es un booleano (calor 0 o 1) que indica si cada punto pertenece al modelo detectado con un determinado umbral de distancia.

El siguiente script implementa la detección de un plano.

In [15]:
# Añade atributo para saber si el punto pertenece o no al plano detectado
is_plane = cloud.add_scalar_field("plane_fit", max_dist=0.1)

#Muestra los puntos de la nube con atributos
cloud.points

,x,y,z,nx(11),ny(11),nz(11),nx(26),ny(26),nz(26),nx,ny,nz,inclination_deg,inclination_deg_10,inclination_deg_25,"voxel_n(V([319 318 319],[0.5 0.5 0.5],True))",is_plane
0,73.017906,7.409195,2.689679,-0.557236,0.318483,0.766849,-0.457854,0.333181,0.824233,-0.457854,0.333181,0.824233,34.489216,39.928261,34.489216,31183379,0
1,72.119446,7.546182,2.660625,-0.557236,0.318483,0.766849,-0.357278,0.361503,0.861202,-0.357278,0.361503,0.861202,30.548214,39.928257,30.548214,30980495,0
2,71.842979,7.746161,2.651559,-0.557236,0.318483,0.766849,-0.357278,0.361503,0.861202,-0.357278,0.361503,0.861202,30.548222,39.928257,30.548222,30879372,0
3,72.741516,8.074124,2.683469,-0.557236,0.318483,0.766849,-0.530865,0.338234,0.777033,-0.530865,0.338234,0.777033,39.010284,39.928261,39.010284,31082256,0
4,72.790054,8.311100,2.685393,-0.557236,0.318483,0.766849,-0.530865,0.338234,0.777033,-0.530865,0.338234,0.777033,39.010284,39.928261,39.010284,31082575,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122744,3.750455,-1.423028,-1.731100,0.407254,-0.141298,0.902319,-0.049804,0.041143,0.997911,-0.049804,0.041143,0.997911,3.703907,25.535421,3.703907,17077190,0
122745,3.744989,-1.408038,-1.726092,0.407253,-0.141298,0.902319,-0.046452,0.050340,0.997651,-0.046452,0.050340,0.997651,3.927701,25.535412,3.927701,17077190,1
122746,3.755518,-1.399045,-1.729087,-0.063138,0.073016,0.995330,-0.053183,0.033867,0.998010,-0.053183,0.033867,0.998010,3.614926,5.539269,3.614926,17077190,0
122747,3.774050,-1.393048,-1.737084,-0.058267,0.072487,0.995666,-0.054668,0.033956,0.997927,-0.054668,0.033956,0.997927,3.689819,5.336342,3.689819,17077190,0


# Tarea 5

Exporta la nube de puntos con la información de la nueva columna que acabas de generar. ¿Cuál es el plano detectado? ¿Puede ser útil la identificación de estos puntos para individualizar objetos? ¿La detección del plano automático, que operación manual realizada con el CloudCompare reemplaza?

Finalmente, vemos que todo el plano detectado no se corresponde exactamente con el elemento. Esto puede ser debido a pequeñas variaciones en altura del suelo, o a que otros puntos influyen en la estimación del plano. No obstante, la mayoría de los errores se corresponden a puntos lejanos que pueden ser eliminados mediante un filtro.

Del mismo modo que en CloudCompare, las librerías para procesado son muy extensas. Aquí hemos visto algunas de sus funciones, y como combinándolas podemos dar solución a problemas complejos, como detectar suelo, esquinas, individualizar objetos, etc. Además, combinadas las librerías de procesado de nubes de puntos con librerías de Inteligencia Artificial, se pueden generar algoritmos para clasificar puntos de forma automática. Esto se verá en la próxima práctica.